In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import matplotlib.ticker as mtick
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
from shapely.geometry import Polygon

import geopandas as gpd
from shapely.geometry import Polygon
from itertools import product
import warnings
warnings.filterwarnings("ignore")

import shapely

In [2]:
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

In [3]:
df.head()

,agricultureStructureIndicator,asOfDate,basementEnclosureCrawlspaceType,policyCount,crsClassificationCode,dateOfLoss,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,baseFloodElevation,ratedFloodZone,houseWorship,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,numberOfFloorsInTheInsuredBuilding,nonProfitIndicator,obstructionType,occupancyType,originalConstructionDate,originalNBDate,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,postFIRMConstructionIndicator,rateMethod,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss,primaryResidenceIndicator,buildingDamageAmount,buildingDeductibleCode,netBuildingPaymentAmount,buildingPropertyValue,causeOfDamage,condominiumCoverageTypeCode,contentsDamageAmount,contentsDeductibleCode,netContentsPaymentAmount,contentsPropertyValue,disasterAssistanceCoverageRequired,eventDesignationNumber,ficoNumber,floodCharacteristicsIndicator,floodWaterDuration,floodproofedIndicator,floodEvent,iccCoverage,netIccPaymentAmount,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,nonPaymentReasonContents,nonPaymentReasonBuilding,numberOfUnits,buildingReplacementCost,contentsReplacementCost,replacementCostBasis,stateOwnedIndicator,waterDepth,floodZoneCurrent,buildingDescriptionCode,rentalPropertyIndicator,state,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
0,0,2020-01-22T16:55:53.000Z,NaN,1,8.0,1998-02-07T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,4.0,0,10.0,1.0,1963-01-01T00:00:00.000Z,1997-01-11T00:00:00.000Z,NaN,NaN,NaN,0,7,0,200000.0,50000.0,1998,0,382.0,0,0.00,937.0,1,N,NaN,0,0.0,NaN,0.0,None,612.0,NaN,0.0,0,Pineapple Express - Southern,15000.0,0.0,60294.0,None,"OCEANSIDE, CITY OF",97.0,1.0,1.0,937.0,NaN,A,0,0.0,None,NaN,0,CA,Currently Unavailable,92056.0,6073.0,6.073019e+09,6.073019e+10,33.2,-117.3,4053d562-94c7-4155-9d60-3030f34a7abc
1,0,2020-01-22T16:55:53.000Z,NaN,1,8.0,2005-08-29T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,2.0,0,NaN,1.0,1967-07-01T00:00:00.000Z,1990-07-12T00:00:00.000Z,NaN,NaN,NaN,0,7,0,100000.0,40000.0,2005,1,NaN,0,0.00,NaN,1,N,NaN,0,0.0,NaN,0.0,None,654.0,NaN,0.0,0,Hurricane Katrina,30000.0,0.0,225203.0,None,NEW ORLEANS/ORLEANS PARISH*,6.0,6.0,1.0,NaN,NaN,A,0,0.0,None,NaN,0,LA,Currently Unavailable,70131.0,22071.0,2.207100e+10,2.207100e+11,29.9,-90.0,4094445e-bc54-42b2-a60d-b949bdc2df3a
2,0,2020-01-22T16:55:53.000Z,NaN,1,9.0,1998-09-28T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,1.0,0,10.0,1.0,1972-01-01T00:00:00.000Z,1997-07-24T00:00:00.000Z,8813.21,1720.0,0.0,0,1,0,100000.0,50000.0,1998,1,9313.0,0,8813.21,80000.0,1,N,2220.0,0,1720.0,0.0,0.0,None,133.0,NaN,0.0,0,Hurricane Georges (Panhandle),15000.0,0.0,120274.0,None,SANTA ROSA COUNTY *,NaN,NaN,1.0,100000.0,0.0,A,0,0.0,None,NaN,0,FL,Currently Unavailable,32566.0,12113.0,1.211301e+10,1.211301e+11,30.4,-86.9,659ba8e0-1bb3-4442-ad57-3f384db43176
3,0,2019-09-19T13:45:58.000Z,1.0,1,9.0,1994-10-07T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,2.0,0,10.0,1.0,1960-01-01T00:00:00.000Z,1993-10-01T00:00:00.000Z,2906.00,0.0,0.0,0,7,0,100000.0,25000.0,1994,0,4428.0,0,2906.00,100000.0,1,N,NaN,0,0.0,NaN,0.0,None,NaN,NaN,0.0,0,None,NaN,0.0,450026.0,None,"BEAUFORT, CITY OF",97.0,NaN,1.0,0.0,NaN,A,0,0.0,None,NaN,0,SC,Currently Unavailable,29902.0,45013.0,4.501300e+10,4.501300e+11,32.4,-80.7,97063578-59b6-403d-b9c3-2e70fd8b6f38
4,0,2019-09-19T13:45:58.000Z,NaN,1,8.0,1996-03-11T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,1.0,0,NaN,1.0,1988-01-01T00:00:00.000Z,1996-01-11T00:00:00.000Z,3875.53,1545.0,0.0,1,7,0,100000.0,25000.0,1996,1,5252.0,0,3875.53,100000.0,1,N,3115.0,0,1545.0,0.0,0.0,None,NaN,NaN,0.0,0,None,NaN,0.0,125092.0,None,BREVARD COUNTY *,NaN,NaN,1.0,0.0,0.0,A,0,0.0,None,1.0,0,FL,Currently Unavailable,32940.0,12009.0,1.200906e+10,1.200906e+11,28.3,-80.7,613c20ef-7838-48ee-a257-07a49e997d3a


In [4]:
#fill na values with 0

df_geographic_unique = df[['state', 'reportedZipCode', 'countyCode', 'censusTract', 'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss']].drop_duplicates()

df_geographic_unique['reportedZipCode'] = df_geographic_unique['reportedZipCode'].fillna(0)
df_geographic_unique['countyCode'] = df_geographic_unique['countyCode'].fillna(0)
df_geographic_unique['censusTract'] = df_geographic_unique['censusTract'].fillna(0)
df_geographic_unique['censusBlockGroupFips'] = df_geographic_unique['censusBlockGroupFips'].fillna(0)
df_geographic_unique['latitude'] = df_geographic_unique['latitude'].fillna(0)
df_geographic_unique['longitude'] = df_geographic_unique['longitude'].fillna(0)

df_geographic_unique = df_geographic_unique.drop_duplicates()

In [5]:
#convert the units to string to use mapping and fix the lenght of each units

df_geographic_unique['reportedZipCode'] = df_geographic_unique['reportedZipCode'].astype(int).astype(str)
df_geographic_unique['reportedZipCode'] = [zipcode.zfill(5) for zipcode in df_geographic_unique['reportedZipCode']]

df_geographic_unique['censusBlockGroupFips'] = [str(int(float(i))) for i in df_geographic_unique['censusBlockGroupFips']]
df_geographic_unique['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_geographic_unique['censusBlockGroupFips']]

df_geographic_unique['countyCode'] = [str(int(float(i))) for i in df_geographic_unique['countyCode']]
df_geographic_unique['countyCode'] = [censusBG.zfill(5) for censusBG in df_geographic_unique['countyCode']]

df_geographic_unique['censusTract'] = [str(int(float(i))) for i in df_geographic_unique['censusTract']]
df_geographic_unique['censusTract'] = [censusBG.zfill(11) for censusBG in df_geographic_unique['censusTract']]

In [6]:
# Define bins and labels for yearOfLoss_1990_2021

bins_1990_2021 = [df_geographic_unique['yearOfLoss'].min(), 2000, 2010, 2020, df_geographic_unique['yearOfLoss'].max() + 1]
labels_1990_2021 = [1990, 2000, 2010, 2020]

df_geographic_unique['yearOfLoss_1990_2021'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_1990_2021, labels=labels_1990_2021, right=False).astype(int)


df_geographic_unique = df_geographic_unique.drop_duplicates()

In [7]:
import pandas as pd

# Step 1: Define the bins.
# Adjusting the bins so that 2011 falls into the 2012 bin
custom_bins = [0, 2000, 2010, 2012] + list(range(2013, 2023)) + [2024]

# Step 2: Define the labels.
# Adjusting the labels to reflect the new bin structure
custom_labels = [0, 2000, 2010] + list(range(2012, 2023))  # The label 2022 will apply to both 2022 and 2023.

# Ensure the number of labels is one less than the number of bin edges.
assert len(custom_bins) == len(custom_labels) + 1, "Number of bin labels must be one fewer than the number of bin edges."

# Step 3: Apply the custom binning and create the 'zip_year_bin' column.
df_geographic_unique['zip_year_bin'] = pd.cut(
    df_geographic_unique['yearOfLoss'],
    bins=custom_bins,
    labels=custom_labels,
    right=False,  # Ensuring the rightmost edge is exclusive.
    include_lowest=True,  # Including the leftmost edge.
).astype(int)  # Converting the resulting categories to integers for convenience.

In [8]:
# Blockgroup shapefile

chunk_size = 40000 
chunks = [x for x in range(0, 120000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/BG_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
BG_df= pd.concat(gdf_list, ignore_index=True)

In [9]:
chunk_size = 50000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 300000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/zipcode_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)
    
# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
zipcode_df = pd.concat(gdf_list, ignore_index=True)

In [10]:
BG_df.rename(columns={'geometry': 'geometry_BG'}, inplace=True)
zipcode_df.rename(columns={'geometry': 'geometry_zipcode'}, inplace=True)

In [11]:
# Initial mapping with multi-index
BG_df.set_index(['GEOID'], inplace=True)
df_geographic_unique['geometry_BG'] = df_geographic_unique.set_index(['censusBlockGroupFips']).index.map(BG_df['geometry_BG'])

BG_df.reset_index(inplace=True)

In [12]:
# Initial mapping with multi-index
zipcode_df.set_index(['ZIPcode', 'year'], inplace=True)
df_geographic_unique['geometry_zipcode'] = df_geographic_unique.set_index(['reportedZipCode', 'zip_year_bin']).index.map(zipcode_df['geometry_zipcode'])

zipcode_df.reset_index(inplace=True)

### Visualization

In [13]:
df_geographic_unique.head()

,state,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss,yearOfLoss_1990_2021,zip_year_bin,geometry_BG,geometry_zipcode
0,CA,92056,06073,06073018512,060730185123,33.2,-117.3,1998,1990,0,"POLYGON ((-117.33340 33.20936, -117.33334 33.2...",None
1,LA,70131,22071,22071000616,220710006163,29.9,-90.0,2005,2000,2000,"POLYGON ((-90.00122 29.92086, -90.00119 29.921...","POLYGON ((-90.01051 29.94005, -90.00980 29.939..."
2,FL,32566,12113,12113010815,121130108152,30.4,-86.9,1998,1990,0,"POLYGON ((-87.03397 30.38297, -87.03395 30.382...",None
3,SC,29902,45013,45013000700,450130007002,32.4,-80.7,1994,1990,0,"POLYGON ((-80.68993 32.43298, -80.68975 32.432...",None
4,FL,32940,12009,12009063107,120090631073,28.3,-80.7,1996,1990,0,"POLYGON ((-80.71440 28.24480, -80.71391 28.244...",None


In [ ]:
# Assuming df_geographic_unique is a DataFrame with a 'geometry_BG' column containing geometry data.
visualize = df_geographic_unique[df_geographic_unique['state'] == 'NC']

# Explicitly create GeoSeries from the 'geometry_BG' column
geometry_bg = gpd.GeoSeries(visualize['geometry_BG'])
geometry_zipcode = gpd.GeoSeries(visualize['geometry_zipcode'])

# Ensure that the GeoSeries has the correct Coordinate Reference System (CRS)
geometry_bg.set_crs("EPSG:4326", inplace=True)
geometry_zipcode.set_crs("EPSG:4326", inplace=True)

# Calculate the centroids for the filtered data
centroids = geometry_bg.centroid

# Apply the centroid-based filter to the DataFrame
visualize_NC_filtered = visualize[(centroids.y < 37) & (centroids.y > 22)]

# Create GeoDataFrames for block groups and zip codes using the filtered DataFrame
gdf_bg = gpd.GeoDataFrame(visualize_NC_filtered, geometry=geometry_bg)
gdf_zipcode = gpd.GeoDataFrame(visualize_NC_filtered, geometry=geometry_zipcode)

# Ensure both GeoDataFrames use the same CRS for overlay operation
gdf_bg = gdf_bg.set_crs(epsg=4326)
gdf_zipcode = gdf_zipcode.set_crs(epsg=4326)

# Calculate the intersection of the two GeoDataFrames
gdf_intersection = gpd.overlay(gdf_bg, gdf_zipcode, how='intersection')

# If you want to plot only the boundaries of the intersection areas
gdf_intersection['borders'] = gdf_intersection.boundary

# Project the intersection to a suitable CRS
gdf_intersection_projected = gdf_intersection.to_crs(epsg=3857)

# Create a base plot
fig, ax = plt.subplots(1, 1, figsize=(10, 10))  # Adjusted for reasonable display size

# Plot the boundaries of intersections
gdf_intersection_projected['borders'].plot(ax=ax, color='green', linewidth=2, label='Overlapping Boundaries')

# Add x, y labels, title, and legend
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Overlap between Block Groups and Zip Codes in NC')
ax.legend()

# Show the plot
plt.show()